## Self Balancing Robot in PyBullet
**Balance and control of a 2-wheeled robot simulated with PyBullet Physics library**
<br>V2: Everything Implemented in GYM 

Importing Required Libraries

In [1]:
import pybullet
import time
import pybullet_data
import numpy as np
import matplotlib.pyplot as plt
import gym
from gym import spaces

GYM Environment For Robot

In [2]:
class SelfBalancing(gym.Env):
    #metadata = {'render.modes': ['human']}

    def __init__(self):
        super(SelfBalancing, self).__init__()
        # Define action and observation space
        self.action_space = spaces.Box(low=0.0, high=+1.0,shape=(3,),dtype=np.float64)
        self.observation_space = spaces.Box(low=np.array([-np.pi/2,-1000]), high=np.array([+np.pi/2,+1000]))
        """
            Action Space: action[0] -> kp, action[1] -> ki, action[2] -> kd
            Observation Space: torso_pitch orientation, torso linear speed
        """
        self.state = np.array([0.0,0.0])
        self.steps = 0
        self.max_episode_steps = 2500
        # Instantiate PyBullet
        phisycsClient = pybullet.connect(pybullet.GUI)
        pybullet.setAdditionalSearchPath(pybullet_data.getDataPath())
        # Spawn Robot
        self.robotID = None
        self.reset()
        # Initialize Controller Parameters
        self.integral = 0
        self.derivative = 0
        self.prev_error = 0
        
    def step(self, action):
        motion = self.controller(action)
        self.take_action(motion)
        ## Calculating reward
        reward = self.calculate_reward()
        obs = self.observe()
        done = self.terminated()
        return obs, reward, done, {}
    
    def take_action(self,motion):
        # Takes a tuple as input
        # motion --> (left wheel speed, right wheel speed)
        pybullet.setJointMotorControl2(bodyUniqueId=self.robotID, 
                        jointIndex=0, 
                        controlMode=pybullet.VELOCITY_CONTROL,
                        targetVelocity = motion[0])
        pybullet.setJointMotorControl2(bodyUniqueId=self.robotID, 
                        jointIndex=1, 
                        controlMode=pybullet.VELOCITY_CONTROL,
                        targetVelocity = motion[1])
        pybullet.stepSimulation()
        time.sleep(1.0/400)
        self.steps += 1
    
    def observe(self):
        position, orientation = pybullet.getBasePositionAndOrientation(self.robotID)
        self.state[0] = np.array([pybullet.getEulerFromQuaternion(orientation)[0]])
        linear_vel, anagular_vel = pybullet.getBaseVelocity(self.robotID)
        self.state[1] = (linear_vel[0]**2 + linear_vel[1]**2 + linear_vel[2]**2) ** 0.5
        return self.state
    
    def calculate_reward(self):
        reward = - (self.observe()[0]**2 + self.observe()[1]**2)
        if self.terminated():
            reward += (self.steps - self.max_episode_steps) / 500
        return reward
    
    def controller(self,action):
        ## Simple PID
        error = self.observe()[0]
        self.integral += error
        self.derivative = error - self.prev_error
        self.prev_error = error
        
        motion = ((action[0]*1000) * error + (action[1]*0.1) * self.integral + (action[2]*100) * self.derivative)
        return (motion,motion)
    
    def reset(self):
        pybullet.resetSimulation()
        planeID = pybullet.loadURDF("plane.urdf")
        pybullet.setGravity(0,0,-9.81)
        theta = np.random.uniform(high = np.pi/4 , low = -np.pi/4)
        self.robotID = pybullet.loadURDF("robot.urdf",
                                 [0.0,0.0,0.0],pybullet.getQuaternionFromEuler([theta,0.0,0.0]),useFixedBase = 0)
        pybullet.setRealTimeSimulation(0) # change to (1) for real time simulation
        self.state = self.observe()
        self.steps = 0
        return self.state
    
    def terminated(self):
        ## If the robot tilt angle reaches 75 degrees or
        ## the simulation reaches its maximum time steps
        if self.steps > self.max_episode_steps or abs(self.state[0]) > (np.pi / (75/80)):
            return True
        else:
            return False
    
    def __del__(self):
        pybullet.disconnect()

Test Environment with (Kp = 650, Ki = 0.005, Kd = 10.0)

In [8]:
env = SelfBalancing()

while not env.terminated():
    env.step((0.55,0.1,0.25))
del env

Advantagous Actor Critic (A2C)

In [3]:
import gym
import coax
import optax
import haiku as hk
import jax.numpy as jnp
import jax
from numpy import prod

# pick environment
name = 'a2c'
env = SelfBalancing()
env = coax.wrappers.TrainMonitor(env, name=name, tensorboard_dir=f"./data/tensorboard/{name}") 

def func_v(S, is_training):
    # custom haiku function
    value = hk.Sequential([
                          hk.Linear(20),
                          hk.Linear(20),
                          hk.Linear(1,w_init=jnp.zeros),jnp.ravel])
    return value(S)  # output shape: (batch_size,)

def func_pi(S, is_training):
    shared = hk.Sequential((
        hk.Linear(20), jax.nn.relu,
        hk.Linear(20), jax.nn.relu,
    ))
    mu = hk.Sequential((
        shared,
        hk.Linear(10), jax.nn.relu,
        hk.Linear(3, w_init=jnp.zeros),
        hk.Reshape(env.action_space.shape),
    ))
    logvar = hk.Sequential((
        shared,
        hk.Linear(8), jax.nn.relu,
        hk.Linear(3, w_init=jnp.zeros),
        hk.Reshape(env.action_space.shape),
    ))
    return {'mu': mu(S), 'logvar': logvar(S)}

# function approximators
v = coax.V(func_v, env)
pi = coax.Policy(func_pi, env)


# specify how to update policy and value function
vanilla_pg = coax.policy_objectives.VanillaPG(pi, optimizer=optax.adam(0.001))
simple_td = coax.td_learning.SimpleTD(v, optimizer=optax.adam(0.002))


# specify how to trace the transitions
tracer = coax.reward_tracing.NStep(n=5, gamma=0.9)
buffer = coax.experience_replay.SimpleReplayBuffer(capacity=256)


for ep in range(10):
    s = env.reset()

    for t in range(env.max_episode_steps):
        a, logp = pi(s, return_logp=True)
        s_next, r, done, info = env.step(a)

        # add transition to buffer
        # N.B. vanilla-pg doesn't use logp but we include it to make it easy to
        # swap in another policy updater that does require it, e.g. ppo-clip
        tracer.add(s, a, r, done, logp)
        while tracer:
            buffer.add(tracer.pop())

        # update
        if len(buffer) == buffer.capacity:
            for _ in range(4 * buffer.capacity // 32):  # ~4 passes
                transition_batch = buffer.sample(batch_size=32)
                metrics_v, td_error = simple_td.update(transition_batch, return_td_error=True)
                metrics_pi = vanilla_pg.update(transition_batch, td_error)
                env.record_metrics(metrics_v)
                env.record_metrics(metrics_pi)

            buffer.clear()

        if done:
            break

        s = s_next
del env

/home/kassra/.local/lib/python3.6/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))
[a2c|MainThread|absl|WARNING] No GPU/TPU found, falling back to CPU. (Set TF_CPP_MIN_LOG_LEVEL=0 and rerun for more info.)
/home/kassra/.local/lib/python3.6/site-packages/jax/_src/numpy/lax_numpy.py:2842: UserWarning: Explicitly requested dtype float64 requested in asarray is not available, and will be truncated to dtype float32. To enable more dtypes, set the jax_enable_x64 configuration option or the JAX_ENABLE_X64 shell environment variable. See https://github.com/google/jax#current-gotchas for more.
  lax._check_user_dtype_supported(dtype, "asarray")
[a2c|MainThread|TrainMonitor|INFO] ep: 1,	T: 2,501,	G: -3.3,	avg_G: 0,	t: 2500,	dt: 11.936ms,	SimpleTD/loss: 0.000429,	VanillaPG/loss: -0.000461
[a2c|MainThread|TrainMonitor|INFO] ep: 2,	T: 5,002,	G: -5.56e+04,	avg_G: 0,	t: 2500,	dt: 10.

Proximal Policy Optimization (PPO)

In [5]:
import gym
import coax
import optax
import haiku as hk
import jax.numpy as jnp
import jax
from numpy import prod

# pick environment
name = 'PPO'
env = SelfBalancing()
env = coax.wrappers.TrainMonitor(env, name=name, tensorboard_dir=f"./data/tensorboard/{name}") 


def func_v(S, is_training):
    # custom haiku function
    value = hk.Sequential([
                          hk.Linear(20),
                          hk.Linear(20),
                          hk.Linear(1,w_init=jnp.zeros),jnp.ravel])
    return value(S)  # output shape: (batch_size,)

def func_pi(S, is_training):
    shared = hk.Sequential((
        hk.Linear(20), jax.nn.relu,
        hk.Linear(20), jax.nn.relu,
    ))
    mu = hk.Sequential((
        shared,
        hk.Linear(3, w_init=jnp.zeros),
        hk.Reshape(env.action_space.shape), jax.nn.sigmoid,
    ))
    logvar = hk.Sequential((
        shared,
        hk.Linear(3, w_init=jnp.zeros),
         hk.Reshape(env.action_space.shape), jax.nn.sigmoid
    ))
    return {'mu': mu(S), 'logvar': logvar(S)}

# function approximators
v = coax.V(func_v, env)
pi = coax.Policy(func_pi, env)


# slow-moving avg of pi
pi_behavior = pi.copy()


# specify how to update policy and value function
ppo_clip = coax.policy_objectives.PPOClip(pi, optimizer=optax.adam(0.001))
simple_td = coax.td_learning.SimpleTD(v, optimizer=optax.adam(0.001))


# specify how to trace the transitions
tracer = coax.reward_tracing.NStep(n=5, gamma=0.9)
buffer = coax.experience_replay.SimpleReplayBuffer(capacity=256)


for ep in range(10):
    s = env.reset()

    for t in range(env.max_episode_steps):
        a, logp = pi_behavior(s, return_logp=True)
        s_next, r, done, info = env.step(a)

        # add transition to buffer
        tracer.add(s, a, r, done, logp)
        while tracer:
            buffer.add(tracer.pop())

        # update
        if len(buffer) == buffer.capacity:
            for _ in range(4 * buffer.capacity // 32):  # ~4 passes
                transition_batch = buffer.sample(batch_size=32)
                metrics_v, td_error = simple_td.update(transition_batch, return_td_error=True)
                metrics_pi = ppo_clip.update(transition_batch, td_error)
                #env.record_metrics(metrics_v)
                #env.record_metrics(metrics_pi)

            buffer.clear()
            pi_behavior.soft_update(pi, tau=0.1)

        if done:
            break

        s = s_next
del env

[a2c|MainThread|TrainMonitor|INFO] ep: 1,	T: 2,501,	G: -60.4,	avg_G: 0,	t: 2500,	dt: 11.208ms
[a2c|MainThread|TrainMonitor|INFO] ep: 2,	T: 5,002,	G: -2.49e+05,	avg_G: 0,	t: 2500,	dt: 10.309ms
[a2c|MainThread|TrainMonitor|INFO] ep: 3,	T: 7,503,	G: -1.36e+03,	avg_G: 0,	t: 2500,	dt: 10.710ms
[a2c|MainThread|TrainMonitor|INFO] ep: 4,	T: 10,004,	G: -507,	avg_G: 0,	t: 2500,	dt: 10.337ms
[a2c|MainThread|TrainMonitor|INFO] ep: 5,	T: 12,505,	G: -113,	avg_G: 0,	t: 2500,	dt: 10.174ms
[a2c|MainThread|TrainMonitor|INFO] ep: 6,	T: 15,006,	G: -57.8,	avg_G: 0,	t: 2500,	dt: 10.293ms
[a2c|MainThread|TrainMonitor|INFO] ep: 7,	T: 17,507,	G: -228,	avg_G: 0,	t: 2500,	dt: 10.219ms
[a2c|MainThread|TrainMonitor|INFO] ep: 8,	T: 20,008,	G: -1.16e+03,	avg_G: 0,	t: 2500,	dt: 10.239ms
[a2c|MainThread|TrainMonitor|INFO] ep: 9,	T: 22,509,	G: -5.37,	avg_G: 0,	t: 2500,	dt: 10.162ms


In [6]:
env = SelfBalancing()
coeff = list('')
for i in np.arange(-np.pi/2,np.pi/2,0.1):
    for j in range(-20,20):
        coeff.append(pi(np.array([i,j]),return_logp=False))
print(np.mean(np.array(coeff),axis=0))
del env

[0.6141107  0.57537735 0.61373246]


Checking Learned Policy

In [40]:
del env
env = SelfBalancing()
while not env.terminated():
    s = env.observe()
    action = pi(s, return_logp=False)
    env.step(action)
del env
print (action)

[0.4427186  0.26619488 0.54129535]


Results<br>
<table>
    <tr>
        <td></td>
        <td>Kp</td>
        <td>Ki</td>
        <td>Kd</td>
    </tr>
    <tr>
        <td>PPO</td>
        <td>614.11</td>
        <td>0.026</td>
        <td>54.13</td>
    </tr>
    <tr>
        <td>A2C</td>
        <td>524.35</td>
        <td>0.049</td>
        <td>51.20</td>
    </tr>
</table>